In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
def cgi_decode(input_data):
    """Decodes the CGI-encoded string "input_data":
       - Replaces "+" by " ".
       - Replaces "%xx" by the character with hex number xx.
       Returns the decoded string.  Raises `ValueError` for invalid input_datas.
       """
    output = ""
    i = 0
    while i < len(input_data):
        current_char = input_data[i]
        if current_char == "+":
            output += " "
        elif current_char == "%":
            digit_high, digit_low = input_data[i + 1], input_data[i + 2]
            i += 2
            try:
                v = int(digit_high, 16) * 16 + int(digit_low, 16)
                output += chr(v)
            except:
                raise ValueError("Invalid input_data")
        else:
            output += current_char
        i += 1
    return output


def line_tracer(frame, event, arg):
    """Traces the collection of lines executed by a function."""
    if event == "line":
        lineno = frame.f_lineno
        global coverage
        coverage.add(lineno)
    return line_tracer


import sys


def record_coverage(function, s):
    """Record the set of lines covered when function calls s."""
    global coverage
    coverage = set([])
    sys.settrace(line_tracer)
    function(s)
    sys.settrace(None)
    coverage = frozenset(coverage)
    return coverage

<IPython.core.display.Javascript object>

In [3]:
class Individual:
    def __init__(self, value):
        self.value = value
        try:
            self.coverage = record_coverage(cgi_decode, self.value)
        except:
            self.coverage = str(sys.exc_info()[0])
            print("Got error " + str(sys.exc_info()[0]) + " for input " + self.value)
        add_to_coverage_dict(self.value, self.coverage)
        self.fitness = None

    def print_info(self):
        print(self.value)
        print(self.coverage)
        print(self.fitness)

    def compute_fitness(self):
        self.fitness = 1 / (coverage_dict[self.coverage])

<IPython.core.display.Javascript object>

In [4]:
important_samples = []
coverage_dict = {}

<IPython.core.display.Javascript object>

In [5]:
def add_to_coverage_dict(value, coverage):
    if coverage_dict.get(coverage, 0) == 0:
        important_samples.append(value)
    coverage_dict[coverage] = coverage_dict.get(coverage, 0) + 1

<IPython.core.display.Javascript object>

In [6]:
import numpy as np

carrying_capacity = 100

<IPython.core.display.Javascript object>

In [7]:
seed = [
    "http://www.google.com/search?q=fuzzing",
    "http://www.google.com/searh;q=fuzzinb2g",
    "http://www.Xgole.com/eaHrh;qfu/zzinb2g",
]

<IPython.core.display.Javascript object>

In [8]:
population = [Individual(x) for x in seed]

<IPython.core.display.Javascript object>

In [9]:
def update_fitness_scores(population):
    for individual in population:
        individual.compute_fitness()

<IPython.core.display.Javascript object>

In [10]:
def print_population_info(population):
    for individual in population:
        individual.print_info()

<IPython.core.display.Javascript object>

In [11]:
import numpy as np


def get_fitness_scores(population):
    fitness_distribution = np.array([x.fitness for x in population])
    fitness_distribution = fitness_distribution / sum(fitness_distribution)
    return fitness_distribution

<IPython.core.display.Javascript object>

In [12]:
import numpy as np


def sample_pair_for_reproduction(population):
    fitness_distribution = get_fitness_scores(population)
    index1 = np.random.choice(len(fitness_distribution), p=fitness_distribution)
    individual1 = population[index1]
    index2 = np.random.choice(len(fitness_distribution), p=fitness_distribution)
    individual2 = population[index2]
    return individual1, individual2

<IPython.core.display.Javascript object>

In [13]:
def recombine(individual1, individual2):
    L1 = len(individual1.value)
    L2 = len(individual2.value)
    L = min(L1, L2)
    locus = np.random.randint(L)
    offspring1_value = individual1.value[:locus] + individual2.value[locus:]
    offspring2_value = individual2.value[:locus] + individual1.value[locus:]
    return Individual(offspring1_value), Individual(offspring2_value)

<IPython.core.display.Javascript object>

In [14]:
update_fitness_scores(population)

<IPython.core.display.Javascript object>

In [15]:
print_population_info(population)

http://www.google.com/search?q=fuzzing
frozenset({7, 8, 9, 10, 11, 13, 22, 23, 24})
0.3333333333333333
http://www.google.com/searh;q=fuzzinb2g
frozenset({7, 8, 9, 10, 11, 13, 22, 23, 24})
0.3333333333333333
http://www.Xgole.com/eaHrh;qfu/zzinb2g
frozenset({7, 8, 9, 10, 11, 13, 22, 23, 24})
0.3333333333333333


<IPython.core.display.Javascript object>

In [16]:
def recombination_phase(population):
    new_population = population.copy()
    population_size = len(population)
    number_of_recombinations = population_size // 2
    for i in range(number_of_recombinations):
        individual1, individual2 = sample_pair_for_reproduction(population)
        try:
            offspring1, offspring2 = recombine(individual1, individual2)
            new_population.append(offspring1)
            new_population.append(offspring2)
        except ValueError:
            pass
    update_fitness_scores(new_population)
    return new_population

<IPython.core.display.Javascript object>

In [17]:
population = recombination_phase(population)

<IPython.core.display.Javascript object>

In [18]:
print_population_info(population)

http://www.google.com/search?q=fuzzing
frozenset({7, 8, 9, 10, 11, 13, 22, 23, 24})
0.2
http://www.google.com/searh;q=fuzzinb2g
frozenset({7, 8, 9, 10, 11, 13, 22, 23, 24})
0.2
http://www.Xgole.com/eaHrh;qfu/zzinb2g
frozenset({7, 8, 9, 10, 11, 13, 22, 23, 24})
0.2
http://www.Xgole.com/eaHrh;qfu/zzinbng
frozenset({7, 8, 9, 10, 11, 13, 22, 23, 24})
0.2
http://www.google.com/search?q=fuzzi2g
frozenset({7, 8, 9, 10, 11, 13, 22, 23, 24})
0.2


<IPython.core.display.Javascript object>

In [20]:
MUTATION_PROBABILITY = 0.1
import random
import string

<IPython.core.display.Javascript object>

In [21]:
def produce_random_character():
    vocabulary = string.punctuation + string.ascii_letters + string.digits
    return random.choice(vocabulary)


def delete_random_character(input_data):
    """Randomly deletes a character in the in file."""
    l = len(input_data)
    position = random.randint(0, l - 1)
    output = input_data[:position] + input_data[(position + 1) :]
    return output


def insert_random_character(input_data):
    """Randomly inserts a character into the in file."""
    output = input_data
    l = len(input_data)
    position = random.randint(0, l - 1)
    inserted_character = produce_random_character()
    output = input_data[:position] + inserted_character + input_data[position:]
    return output


def swap_two_characters(input_data):
    """Swap two characters at random."""
    l = len(input_data)
    position1 = random.randint(0, l - 1)
    character1 = input_data[position1]
    position2 = random.randint(0, l - 1)
    character2 = input_data[position2]
    if position1 == position2:
        return input_data
    min_position = min(position1, position2)
    max_position = max(position1, position2)
    output = (
        input_data[:min_position]
        + input_data[max_position]
        + input_data[(min_position + 1) : (max_position)]
        + input_data[min_position]
        + input_data[(max_position + 1) :]
    )
    return output


def randomize_character(input_data):
    """Randomly changes a character in the in file."""
    l = len(input_data)
    position = random.randint(0, l - 1)
    character_after = produce_random_character()
    output = input_data[:position] + character_after + input_data[(position + 1) :]
    return output

<IPython.core.display.Javascript object>

In [22]:
def mutate(input_data, num_mutations=3):
    """Perform num_mutations many mutations on in."""
    output = input_data
    mutation_operations = [
        randomize_character,
        insert_random_character,
        delete_random_character,
        swap_two_characters,
    ]
    for i in range(num_mutations):
        current_mutation_operation = random.choice(mutation_operations)
        output = current_mutation_operation(output)
    return output

<IPython.core.display.Javascript object>

In [23]:
def mutation_phase(population):
    for i in range(len(population)):
        individual = population[i]
        mutate_this_individual = np.random.choice(
            2, p=[1 - MUTATION_PROBABILITY, MUTATION_PROBABILITY]
        )
        if mutate_this_individual:
            try:
                mutated_individual = Individual(mutate(individual.value))
                population[i] = mutated_individual
            except ValueError:
                pass
    update_fitness_scores(population)
    return population

<IPython.core.display.Javascript object>

In [24]:
population = mutation_phase(population)

<IPython.core.display.Javascript object>

In [25]:
print_population_info(population)

http://www.oogle.co/search?q=fuzzig
frozenset({7, 8, 9, 10, 11, 13, 22, 23, 24})
0.16666666666666666
http://www.google.com/searh;q=fuzzinb2g
frozenset({7, 8, 9, 10, 11, 13, 22, 23, 24})
0.16666666666666666
http://www.Xgole.com/eaHrh;qfu/zzinb2g
frozenset({7, 8, 9, 10, 11, 13, 22, 23, 24})
0.16666666666666666
http://www.Xgole.com/eaHrh;qfu/zzinbng
frozenset({7, 8, 9, 10, 11, 13, 22, 23, 24})
0.16666666666666666
http://www.google.com/search?q=fuzzi2g
frozenset({7, 8, 9, 10, 11, 13, 22, 23, 24})
0.16666666666666666


<IPython.core.display.Javascript object>

In [26]:
def culling_phase(population):
    population_size = len(population)
    N = min(carrying_capacity, population_size)
    fitness_distribution = get_fitness_scores(population)
    new_population = list(np.random.choice(population, N, p=fitness_distribution))
    update_fitness_scores(new_population)
    return new_population

<IPython.core.display.Javascript object>

In [27]:
population = culling_phase(population)

<IPython.core.display.Javascript object>

In [28]:
print_population_info(population)

http://www.oogle.co/search?q=fuzzig
frozenset({7, 8, 9, 10, 11, 13, 22, 23, 24})
0.16666666666666666
http://www.Xgole.com/eaHrh;qfu/zzinbng
frozenset({7, 8, 9, 10, 11, 13, 22, 23, 24})
0.16666666666666666
http://www.oogle.co/search?q=fuzzig
frozenset({7, 8, 9, 10, 11, 13, 22, 23, 24})
0.16666666666666666
http://www.google.com/searh;q=fuzzinb2g
frozenset({7, 8, 9, 10, 11, 13, 22, 23, 24})
0.16666666666666666
http://www.oogle.co/search?q=fuzzig
frozenset({7, 8, 9, 10, 11, 13, 22, 23, 24})
0.16666666666666666


<IPython.core.display.Javascript object>

In [29]:
number_of_generations = 50
for g in range(number_of_generations):
    print("generation " + str(g))
    population = recombination_phase(population)
    population = mutation_phase(population)
    population = culling_phase(population)

generation 0
generation 1
generation 2
generation 3
generation 4
generation 5
Got error <class 'ValueError'> for input httH://%wwioogle.com/eaprh?q=fuzz.g
generation 6
Got error <class 'ValueError'> for input httH://%wwioogle.com/eaprh?q=fuzz.g
Got error <class 'ValueError'> for input httH://%wwioogle.com/eaprh?q=fuzz.g
Got error <class 'ValueError'> for input htcH://%wwioogle.com/eaprh?q=fuzz.g
Got error <class 'ValueError'> for input httH://%wwioogle.com/eaprh?q=fuzz.g
Got error <class 'ValueError'> for input httH://%wwioogle.com/eaprh?q=fuzz.g
Got error <class 'ValueError'> for input httH://%wwioogle.com/eaprh?q=fuzz.g
Got error <class 'ValueError'> for input httH://%wwioogle.com/eaprh?q=fuzz.g
Got error <class 'ValueError'> for input httH://%wwioogle.com/eaprh?q=fuzz.g
Got error <class 'ValueError'> for input httH://%ww.oogle.zo/search?0=fuztig
Got error <class 'ValueError'> for input httH://%wwioogle.com/eaprh?q=fuztig
Got error <class 'ValueError'> for input httH://%wwioogle.com/

Got error <class 'ValueError'> for input httH://%wwioogle.com/eaprh?q=fuzz.g
Got error <class 'ValueError'> for input httH://%wwioogle.com/eaprh?q=fuzz.g
Got error <class 'ValueError'> for input http://%wwioogle.comsearch?0=fuztig
Got error <class 'ValueError'> for input wtth///%wwioogle.com/eaprh?q=fuzz.g
Got error <class 'ValueError'> for input httH://%wwioogle.com/earch?0=fuztig
Got error <class 'ValueError'> for input httH://%wwioogle.comsearch?0=fuztig
Got error <class 'ValueError'> for input httH://%wwioogle.com/earh?q=fzz.g
Got error <class 'ValueError'> for input httH://%wwioogle.com/eaprh?q=fuzzig
Got error <class 'ValueError'> for input httH://%wwioogle.com/eaprh?q=fuzz.g
Got error <class 'ValueError'> for input httH://%wwiooqul.zo/search?0=fuztig
Got error <class 'ValueError'> for input httH://%wwiooqul_e.com/aeprh?3g=fuzzg
Got error <class 'ValueError'> for input ttH://%w&wioogle.coi/earch?0=fuztmg
Got error <class 'ValueError'> for input httH:/%wwio5ogle.gcom/eaprh?q=fuzz.

Got error <class 'IndexError'> for input h.tcp:U/www.Cog?+.zo/searchlr0=fuR%}
Got error <class 'ValueError'> for input htcp:U/www.Cogl+}z0/sraroc?%oc=fuRti.
Got error <class 'ValueError'> for input htcp:U/www.Cogl+.zo/srarc?0%O=fRuti}
Got error <class 'ValueError'> for input htcp:U/www.Cogl+.zo/s"arc?%c=futi}
Got error <class 'ValueError'> for input htcp:U/www.CIgl+.zo/srrac?f0c=%uRti}
Got error <class 'ValueError'> for input htcp:U/wwwwCogl+.zo/srarc?%i0c=fu\ti}
generation 33
Got error <class 'ValueError'> for input htcp:U/www.Jogl6zo/srarc?%0%0c=fuRti
Got error <class 'ValueError'> for input htcp:U/wwqw.CoglK'.zo/srarc?%-c=fbRti
Got error <class 'ValueError'> for input tcpU8/wwoww.Jwgl6zo/srarc?=0c%fuRti}
Got error <class 'ValueError'> for input htcp:U/www%.qJogl6zo/Crarc?%0c=fuRti}
Got error <class 'ValueError'> for input htcp:%U/www.Jogl6z/srarc?%0&=fuRti}
Got error <class 'ValueError'> for input %tcp:U/wwqw.ogl6zo/srarc?h}cAfuRti0
Got error <class 'ValueError'> for input htQp:U/ww

Got error <class 'ValueError'> for input htcp:Uuwww.Jogl6zo/oas//rc?%0=fuRti}
Got error <class 'ValueError'> for input htcp:Uuwww.Jogl6zo/oas//rc?%c=fuRti}
Got error <class 'ValueError'> for input htcp:U/www.Jogl6zo/srarc?%0=fuRti}
Got error <class 'ValueError'> for input htcp:U/www.Jogl6zo/raarc?%0=fuRti}
Got error <class 'ValueError'> for input htcp:U/www.Jogl6zo/srarc?f0cR%mu=ti}
Got error <class 'ValueError'> for input htcp:Uwww.Jogl6zo/raa0c?%rc=fuRti}
Got error <class 'ValueError'> for input htcp:www.Cogl+:so/zrarc?%c=fuRti0
Got error <class 'ValueError'> for input htcR:/www.Jog%6zo/raarc}?0c=fupti
Got error <class 'ValueError'> for input htcp:cww.Cogl+:zo/srarh?%y0ccw=fuRti}
generation 47
Got error <class 'ValueError'> for input htcp:U/www.Jogl6zo/srarc?%c=fuRti}
Got error <class 'ValueError'> for input htcTp:www.Cogl+:zo/srac?%}c=fuRti0
Got error <class 'ValueError'> for input htcp:www.Cogl+:zoracc?%0r=fuRti}
Got error <class 'ValueError'> for input htc0:U/www.Jognl6zo/srarc?%p

<IPython.core.display.Javascript object>

In [30]:
important_samples

['http://www.google.com/search?q=fuzzing',
 'httH://%wwioogle.com/eaprh?q=fuzz.g',
 'htcp://www.oogle.zo/search?0=fuztig',
 'htcp://www.oogl+.zo/search?r0=fuzti}',
 'htc+:/?www.oogle.zo/search/0=fujztig',
 'htcp:U/www.ogl+.zC/search?r0=fuRt%}',
 'htcp:U/www.Cogl+.zo/srarc?%0c=fuRti}',
 'tcpU/www.Cogl+:zo/srarc?%0c=fuRti}',
 'htcp:U/www.Jogl6zo/srarc?%0c=fuRti}',
 'htcp:U/wwqw.CoglK.zo/srarc?%0c=fuRti']

<IPython.core.display.Javascript object>

In [31]:
coverage_dict

{frozenset({7, 8, 9, 10, 11, 13, 22, 23, 24}): 634,
 "<class 'ValueError'>": 293,
 frozenset({7, 8, 9, 10, 11, 13, 22, 23, 24, 41, 42, 43}): 112,
 frozenset({7, 8, 9, 10, 11, 12, 13, 22, 23, 24}): 2394,
 frozenset({7, 8, 9, 10, 11, 12, 13, 22, 23, 24, 41, 42, 43}): 97,
 "<class 'IndexError'>": 115,
 frozenset({7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 22, 23, 24}): 643,
 frozenset({7,
            8,
            9,
            10,
            11,
            12,
            13,
            14,
            15,
            16,
            17,
            18,
            22,
            23,
            24,
            41,
            42,
            43}): 38,
 frozenset({7,
            8,
            9,
            10,
            11,
            13,
            14,
            15,
            16,
            17,
            18,
            22,
            23,
            24,
            41,
            42,
            43}): 54,
 frozenset({7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 18, 22, 2

<IPython.core.display.Javascript object>

In [35]:
cgi_decode("htcp:U/www.ooglez/search/0=fujuRt%")

IndexError: string index out of range

<IPython.core.display.Javascript object>